# Customization & Components

## Customize the activation function

Modification based on [Ref](https://medium.com/@chinesh4/custom-activation-function-in-tensorflow-for-deep-neural-networks-from-scratch-tutorial-b12e00652e24)

In [2]:
# !pip install tensorflow --upgrade

In [ ]:
customize activation    

#work/python/keras-1

#work/DNN/Comparison-activation

In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [4]:
tf.__version__

'2.4.1'

### Example 1

[Gaussin Radial Basis Function](https://en.wikipedia.org/wiki/Radial_basis_function) is defined as
$$
\phi(x) = e^{-(\epsilon r)^2}
$$

We found out that radial basis function works pretty good as an activation in a network, this will be demonstrated in the following tutorials. The raial basis function only makes a difference in an interval close to the center which is similar to the idea of "on-and-off" of an activation function.

In a dimension 1 space, let us set activation to be $\phi(x)=e^{-x^2}$. Since this is not in TensorFlow's list of activation functions https://www.tensorflow.org/api_docs/python/tf/keras/activations, we will use it for demonstration of customizing activation function.

In [2]:
#######################################
#define the activation function
def rbf(x):
    return tf.math.exp(-x**2)

#######################################
#define the derivative of the activation function
def d_rbf(x):
    return tf.gradients(rbf,x)

#######################################
#we couldn't use “d_rbf” as an activation function if we wanted to 
#because tensorflow doesn't know how to calculate the gradients of that function.
def rbf_grad(op, grad):
    x = op.inputs[0]
    n_gr = d_rbf(x)    #defining the gradient.
    return grad * n_gr

def py_func(func, inp, Tout, stateful=True, name=None, grad=None):
    # Need to generate a unique name to avoid duplicates:
    rnd_name = 'PyFuncGrad' + str(np.random.randint(0, 1E+2))
    tf.RegisterGradient(rnd_name)(grad)
    g = tf.get_default_graph()
    with g.gradient_override_map({"PyFunc": rnd_name, "PyFuncStateless": rnd_name}):
        return tf.py_func(func, inp, Tout, stateful=stateful, name=name)
    
def tf_rbf(x,name=None):
    with tf.name_scope(name, "rbf", [x]) as name:
        y = py_func(rbf,   #forward pass function
                    [x],
                    [tf.float32],
                    name=name,
                    grad= rbf_grad) #the function that overrides gradient
        y[0].set_shape(x.get_shape())     #when using with the code, it is used to specify the rank of the input.
    return y[0]

In [3]:
## Training data
np.random.seed(1)
x_train = np.linspace(-40,40,20)
y_train = 0.3*x_train**2 + np.random.normal(0, 1, len(x_train)) 

In [4]:
model_rbf = tf.keras.Sequential()
model_rbf.add(tf.keras.layers.Dense(3,activation=rbf))     #no need to put quote around rbf
model_rbf.add(tf.keras.layers.Dense(1))

model_rbf.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(0.1))
model_rbf.fit(x_train,y_train, epochs=1000, verbose=0)

# model.summary()
model_rbf.get_weights()   #test if model is trained successfully

[array([[-2.6122294e-03,  1.6906145e-03, -2.2995217e+00]], dtype=float32),
 array([-4.2092835e-04,  2.8977118e-04,  4.8410983e+00], dtype=float32),
 array([[  57.36145 ],
        [  59.288876],
        [-115.397705]], dtype=float32),
 array([55.409958], dtype=float32)]

## Cutomize the loss function 

In [9]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(3,activation=rbf))     #no need to put quote around rbf
model.add(tf.keras.layers.Dense(1))

In [12]:
# customize loss
def custom_loss(ytrue,ypred):
    val = tf.math.reduce_mean(tf.math.square((ytrue-ypred)/10))
    return val

In [16]:
model.compile(loss=custom_loss,optimizer=tf.keras.optimizers.Adam(0.01))
model.fit(x_train,y_train,epochs=100,verbose=0)

model.get_weights()    #test if model is trained successfully

[array([[ 4.8343139e-05, -1.8820183e-01,  5.0299735e-05]], dtype=float32),
 array([1.9036455e-05, 5.2888556e-03, 2.9459905e-05], dtype=float32),
 array([[ 5.030023 ],
        [-4.4802723],
        [ 4.8218493]], dtype=float32),
 array([3.9862227], dtype=float32)]

- [Models](#Models)
- [Layers](#Layers)
- [Activations]
- [Fully Connected Networks]

## Components
### Models

In [5]:
import time
import numpy as np
import pandas as pd
import tensorflow as tf
from functools import reduce
from matplotlib import pyplot as plt
from pprint import pprint
print(tf.__version__)
tf.random.set_seed(42)
true_weights = tf.constant(list(range(5)), dtype=tf.float32)[:, tf.newaxis]
x = tf.constant(tf.random.uniform((32, 5)), dtype=tf.float32)
y = tf.constant(x @ true_weights, dtype=tf.float32)

2.4.1


In [6]:
weights = tf.Variable(tf.random.uniform((5, 1)), dtype=tf.float32)
y_hat = tf.linalg.matmul(x, weights)

In [6]:
# np.all(tf.matmul(x, weights) == tf.linalg.matmul(x, weights))

True

Recall our linear regression model in Tutorial4, We can write a simple class to do the job.

In [7]:
class LinearRegression(object):
    def __init__(self, num_parameters):
        self._weights = tf.Variable(tf.random.uniform((num_parameters,1)), dtype=tf.float32)
    
    @tf.function
    def __call__(self, x):
        return tf.linalg.matmul(x, self._weights)
    
    @property
    def variables(self):
        return self._weights

In [17]:
# model = LinearRegression(5)
# model(x)

Note that we decorated the `__call__` method with `tf.function`, hence a graph will
be generated to back up the computation.

With this class, we can rewrite the training code as:

In [8]:
model = LinearRegression(5)

@tf.function
def train_step():
    with tf.GradientTape() as tape:
        y_hat = model(x)
        loss = tf.reduce_mean(tf.square(y - y_hat))
    gradients = tape.gradient(loss, model.variables)
    model.variables.assign_add(tf.constant([-0.05], dtype=tf.float32)* gradients)
    return loss

t0 = time.time()
for iteration in range(1001):
    loss = train_step()
    if not (iteration % 200):
        print('mean squared loss at iteration {:4d} is {:5.4f}'.format(iteration, loss))

pprint(model.variables)
print('time took: {} seconds'.format(time.time() - t0))

mean squared loss at iteration    0 is 18.7201
mean squared loss at iteration  200 is 0.0325
mean squared loss at iteration  400 is 0.0017
mean squared loss at iteration  600 is 0.0002
mean squared loss at iteration  800 is 0.0000
mean squared loss at iteration 1000 is 0.0000
<tf.Variable 'Variable:0' shape=(5, 1) dtype=float32, numpy=
array([[-3.1365452e-03],
       [ 1.0065703e+00],
       [ 2.0000944e+00],
       [ 3.0032609e+00],
       [ 3.9934685e+00]], dtype=float32)>
time took: 0.39346790313720703 seconds


Here, we still decorated the `train_step` function with `tf.function`. The reason is
that <font color=red>the loss and gradient calculation can also benefit from graphs</font>.
This simple model class works fine. But it can be better if we subclass from
`tf.keras.Model`.

In [11]:
class LinearRegression(tf.keras.Model):
    def __init__(self, num_parameters, **kwargs):
        super().__init__(**kwargs)
        self._weights = tf.Variable(tf.random.uniform((num_parameters,1)), dtype=tf.float32)
    
    @tf.function
    def call(self, x):
        return tf.linalg.matmul(x, self._weights)

This model class has a few differences compare with the organic version. First is that we
implemented the call method rather than the dunder version of it. Under the hood,
`tf.keras.Model`'s `__call__` method is a wrapper over this `call` method, and it is
performing, among many other things, things like converting inputs to tensors and graph
building. The second thing is that we dropped the accessor for the variables because we
inherited one.


With this subclass model, we need to modify the training code a bit to make it work. The
`.variables` accessor from `tf.keras.Model` gives us a collection of references to
the model's variables, to accommodate complex models with many sets of variables. So,
as a result, the corresponding gradients will be a collection too.

In [13]:
model = LinearRegression(5)
# model(x)
model.get_weights()   #only the value in model.variables

[array([[0.7402308 ],
        [0.33938193],
        [0.5692506 ],
        [0.44811392],
        [0.29285502]], dtype=float32)]

In [14]:
model.variables   #this is a list

[<tf.Variable 'Variable:0' shape=(5, 1) dtype=float32, numpy=
 array([[0.7402308 ],
        [0.33938193],
        [0.5692506 ],
        [0.44811392],
        [0.29285502]], dtype=float32)>]

In [16]:
model = LinearRegression(5)

@tf.function
def train_step():
    with tf.GradientTape() as tape:
        y_hat = model(x)
        loss = tf.reduce_mean(tf.square(y - y_hat))
    gradients = tape.gradient(loss, model.variables)
    for g, v in zip(gradients, model.variables):      #if use the above way to update variables, error; see below
        v.assign_add(tf.constant([-0.05], dtype=tf.float32) * g)
    return loss

t0 = time.time()
for iteration in range(1001):
    loss = train_step()
    if not (iteration % 200):
        print('mean squared loss at iteration {:4d} is {:5.4f}'.format(iteration, loss))
        
pprint(model.variables)
print('time took: {} seconds'.format(time.time() - t0))

mean squared loss at iteration    0 is 14.3411
mean squared loss at iteration  200 is 0.0294
mean squared loss at iteration  400 is 0.0016
mean squared loss at iteration  600 is 0.0002
mean squared loss at iteration  800 is 0.0000
mean squared loss at iteration 1000 is 0.0000
[<tf.Variable 'Variable:0' shape=(5, 1) dtype=float32, numpy=
array([[-3.2304246e-03],
       [ 1.0066777e+00],
       [ 2.0000989e+00],
       [ 3.0033290e+00],
       [ 3.9933815e+00]], dtype=float32)>]
time took: 0.22827982902526855 seconds


In [54]:
## Uncomment the following to see the error message

# model = LinearRegression(5)

# @tf.function
# def train_step():
#     with tf.GradientTape() as tape:
#         y_hat = model(x)
#         loss = tf.reduce_mean(tf.square(y - y_hat))
#     gradients = tape.gradient(loss, model.variables)    #model.variables is a list
#     model.variables.assign_add(tf.constant([-0.05], dtype=tf.float32) * gradients)
#     return loss

# t0 = time.time()
# for iteration in range(1001):
#     loss = train_step()
#     if not (iteration % 200):
#         print('mean squared loss at iteration {:4d} is {:5.4f}'.format(iteration, loss))
        
# pprint(model.variables)
# print('time took: {} seconds'.format(time.time() - t0))

Through subclassing `tf.keras.Model` , we get to use many methods from it, like print
out a summary and the Keras model training/testing methods.

In [24]:
print(model.summary())
model.compile(loss='mse', metrics=['mae'])
print(model.evaluate(x, y, verbose=-1))    

Model: "linear_regression_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________
None
[4.5830074668629095e-06, 0.0017235130071640015]


In [26]:
## MSE
tf.reduce_mean(tf.square(model(x) - y))

<tf.Tensor: shape=(), dtype=float32, numpy=4.5830075e-06>

In [28]:
## MAE
tf.reduce_mean(tf.math.abs(model(x) - y))

<tf.Tensor: shape=(), dtype=float32, numpy=0.001723513>

Let's also try using the `.fit` API to train our linear regression model.

In [29]:
model = LinearRegression(5)
model.compile(optimizer='SGD', loss='mse')
model.optimizer.lr.assign(.05)
t0 = time.time()
history = model.fit(x, y, epochs=1001, verbose=0)
pprint(history.history['loss'][::200])
pprint(model.variables)
print('time took: {} seconds'.format(time.time() - t0))

[18.954912185668945,
 0.02976365201175213,
 0.0010764360195025802,
 3.935765562346205e-05,
 1.450204877073702e-06,
 5.471034825177412e-08]
[<tf.Variable 'Variable:0' shape=(5, 1) dtype=float32, numpy=
array([[5.6911039e-04],
       [1.0001738e+00],
       [1.9999688e+00],
       [2.9999278e+00],
       [3.9994378e+00]], dtype=float32)>]
time took: 0.9272251129150391 seconds


Not surprisingly we recovered the ground truth, but it took significantly more time
compared to our custom training loop, it is probably due to the convenient `.fit` API is
doing a bunch more stuff than updating parameters.

Now, let's spice up the model a bit by adding a additional useless bias term and initialize it
with a large value. Ideally, after training, this bias term would become an insignificantly
small number.

In [48]:
class LinearRegressionV2(tf.keras.Model):
    def __init__(self, num_parameters, **kwargs):
        super().__init__(**kwargs)
        self._weights = tf.Variable(tf.random.uniform((num_parameters,1)), dtype=tf.float32)
        self._bias = tf.Variable([100], dtype=tf.float32)
    
    @tf.function
    def call(self, x):
        return tf.linalg.matmul(x, self._weights) + self._bias
    
model = LinearRegressionV2(5)
@tf.function
def train_step():
    with tf.GradientTape() as tape:
        y_hat = model(x)      ##here is the key!!!
        loss = tf.reduce_mean(tf.square(y - y_hat))
    gradients = tape.gradient(loss, model.variables)
    for g, v in zip(gradients, model.variables):      #if use the above way to update variables, error; see below
        v.assign_add(tf.constant([-0.05], dtype=tf.float32) * g)
    return loss

t0 = time.time()
for iteration in range(1001):
    loss = train_step()

pprint(model.variables)

[<tf.Variable 'Variable:0' shape=(5, 1) dtype=float32, numpy=
array([[-2.8140063],
       [-4.8734136],
       [-2.0642996],
       [-1.3286525],
       [ 2.171379 ]], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([9.778882], dtype=float32)>]


Hmm, something is very wrong, the bias term is not updated at all. Guess where might be
the problem? It's in the `train_step` function. Since the input signature has not
changed(there is none), the function is been lazy and did not realize it should do a
retracing. Thus it is training with the old graph, in which the bias term simply does not
exist!

To address this issue, we can make `model` as an input to `train_step`, so that when
the function is invoked with a different model it will create or grab a graph accordingly.

In [49]:
@tf.function
def train_step(model):
    with tf.GradientTape() as tape:
        y_hat = model(x)
        loss = tf.reduce_mean(tf.square(y - y_hat))
    gradients = tape.gradient(loss, model.variables)
    for g, v in zip(gradients, model.variables):
        v.assign_add(tf.constant([-0.05], dtype=tf.float32) * g)
    return loss

model = LinearRegression(5)
for iteration in range(1001):
    loss = train_step(model)
pprint(model.variables)

model = LinearRegressionV2(5)
for iteration in range(5001):
    loss = train_step(model)
pprint(model.variables)

print(train_step._get_tracing_count())

[<tf.Variable 'Variable:0' shape=(5, 1) dtype=float32, numpy=
array([[-3.5654954e-03],
       [ 1.0072978e+00],
       [ 2.0001168e+00],
       [ 3.0036411e+00],
       [ 3.9927874e+00]], dtype=float32)>]
[<tf.Variable 'Variable:0' shape=(5, 1) dtype=float32, numpy=
array([[-2.6677379e-03],
       [ 9.9405426e-01],
       [ 1.9960008e+00],
       [ 2.9956532e+00],
       [ 3.9983604e+00]], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.00961742], dtype=float32)>]
2


### Layers

We typically use the Model class for the overall model architecture, that is how may
combine many smaller computational units to do the job. It is ok to code up small units
with `tf.keras.Model` and then combine them, but since we won't really need many of
the model specific functionalities with these smaller building blocks(for example, its
unlikely we would ever want to call the .fit method on a unit). It is better to use the
`tf.keras.layers.Layer` class. Actually, the Model class is a wrapper over the Layer
class.

Lets say that we want to 'upgrade' the linear regression model to be a composition of a
few linear transformations. We can code up the linear transformation as a Layer class and
then just combine a bunch of its instances of it them in one model.

In [52]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, num_inputs, num_outputs, **kwargs):
        super().__init__(**kwargs)
        self._weights = tf.Variable(tf.random.uniform((num_inputs, num_outputs)), dtype=tf.float32)
    
    @tf.function
    def call(self, x):
        return tf.linalg.matmul(x, self._weights)

class Regression(tf.keras.Model):
    def __init__(self, num_inputs_per_layer, num_outputs_per_layer, **kwargs):
        super().__init__(**kwargs)
        self._layers = [Linear(num_inputs, num_outputs) for (num_inputs, num_outputs) in zip(num_inputs_per_layer, num_outputs_per_layer)]
    
    @tf.function
    def call(self, x):
        for layer in self._layers:
            x = layer(x)
        return x

In `Linear` class definition above, we swapped the super class and generalized it with
the option to specify output size. In the `Regression` model class, we have the option to
use one or a chain of this Linear layers. One obvious benefit with this set up, is that we
now separated the concern of how individual computing units should work and the overall
architecture design of the model. We use Layer to handle the first, and use Model to
tackle the latter.

Let's see if the model is trainable.

In [53]:
model = Regression([5, 3], [3, 1])
for iteration in range(1001):
    loss = train_step(model)
print('Mean absolute error is: ', tf.reduce_mean(tf.abs(y - model(x))).numpy())

Mean absolute error is:  1.3485551e-06


One problem with this linear layer is that it needs the complete sizing information and
allocates resources for all the variables upfront. Ideally, we want it to be a bit lazy, it
should calculate variable sizes and occupy resources only when needed. To archive this,
we implement the `build` method which will handle the variable initialization. The
`build` method can be explicitly called, or it will be invoked automatically the first time
there is data flow to it. With this, the constructor now only stores the hyperparameters for
the layer.

In [70]:
### Linear!!! Build!!!
class Linear(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super(Linear, self).__init__(**kwargs)
        self.units = units
        
    def build(self, input_shape):
        self._weights = self.add_weight(shape=(input_shape[-1], self.units))  #for the variable
        super().build(input_shape)
        
    @tf.function
    def call(self, x):
        output = tf.linalg.matmul(x, self._weights)
        return output

class Regression(tf.keras.Model):
    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)
        self._layers = [Linear(unit) for unit in units]

    @tf.function
    def call(self, x):
        for layer in self._layers:
            x = layer(x)
        return x

model = Regression([3, 2, 1])

pprint(model.variables) # should be empty

for iteration in range(1001):
    loss = train_step(model)

print('Mean absolute error is: ', tf.reduce_mean(tf.abs(y - model(x))).numpy())
pprint(model.variables)

[]
Mean absolute error is:  2.0525858
[<tf.Variable 'linear_16/Variable:0' shape=(5, 3) dtype=float32, numpy=
array([[ 0.37567663, -0.07886422,  0.40833658],
       [ 0.02583691, -0.1018228 , -0.26026365],
       [ 0.23467508, -0.30758873, -0.51971763],
       [ 0.71906245, -0.02258762, -0.66875553],
       [ 0.33473828,  0.04685464, -1.3837334 ]], dtype=float32)>,
 <tf.Variable 'linear_17/Variable:0' shape=(3, 2) dtype=float32, numpy=
array([[-0.22774057, -0.97255296],
       [ 0.19852304,  0.31224003],
       [-0.92619157,  0.8598225 ]], dtype=float32)>,
 <tf.Variable 'linear_18/Variable:0' shape=(2, 1) dtype=float32, numpy=
array([[ 0.61150837],
       [-1.285613  ]], dtype=float32)>]


Tensorflow has a lot of layer options, we will cover a sample of them in later application
specific chapters. For now, we will just quickly see if the linear layer(called Dense ) from
Tensorflow works the same.

In [71]:
class Regression(tf.keras.Model):
    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)
        self._layers = [tf.keras.layers.Dense(unit, use_bias=False) for unit in units] # the only change

    @tf.function
    def call(self, x):
        for layer in self._layers:
            x = layer(x)
        return x

model = Regression([3,2,1])
for iteration in range(1001):
    loss = train_step(model)
    
print('Mean absolute error is: ', tf.reduce_mean(tf.abs(y - model(x))).numpy())
pprint(model.variables)

Mean absolute error is:  0.41030595
[<tf.Variable 'dense_2/kernel:0' shape=(5, 3) dtype=float32, numpy=
array([[ 0.1362488 ,  0.46159887, -0.36640394],
       [-0.57381153,  0.0304967 , -0.1651423 ],
       [ 0.29426968, -1.1551083 , -0.40692037],
       [-0.34422547, -0.28362918, -1.0632066 ],
       [-1.1095847 , -0.76221883, -0.7997634 ]], dtype=float32)>,
 <tf.Variable 'dense_3/kernel:0' shape=(3, 2) dtype=float32, numpy=
array([[ 0.85698915, -0.54857117],
       [ 0.80958056,  0.13226742],
       [ 1.2272625 ,  0.5923777 ]], dtype=float32)>,
 <tf.Variable 'dense_4/kernel:0' shape=(2, 1) dtype=float32, numpy=
array([[-1.6973131 ],
       [-0.27042937]], dtype=float32)>]


### Activations

So our newest model is a composition of two linear transformation, but the composition of
two linear transformations is just another linear transformation.

In [72]:
reduced_model = reduce(tf.linalg.matmul, model.variables)  #apply tf.linalg.matmul to the elements in model.variables
print(reduced_model)
print(tf.reduce_all(tf.abs(model(x) - x @ reduced_model) < 1e-6))

tf.Tensor(
[[-0.00683768]
 [ 1.0769873 ]
 [ 2.1570036 ]
 [ 3.2345533 ]
 [ 4.318075  ]], shape=(5, 1), dtype=float32)
tf.Tensor(False, shape=(), dtype=bool)


In [74]:
model.variables[0] @ model.variables[1] @ model.variables[2]

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[-0.00683768],
       [ 1.0769873 ],
       [ 2.1570036 ],
       [ 3.2345533 ],
       [ 4.318075  ]], dtype=float32)>

Without anything interesting in between, this is just adding unnecessary complexity. The
simplest thing to do is to add some non-linear in-place transformation to the intermediate
results, and this is call activations.

Let's add activations as layers.

In [75]:
class ReLU(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    @tf.function
    def call(self, x):
        return tf.maximum(tf.constant(0, x.dtype), x)
    
class NeuralNetwork(tf.keras.Model):
    def __init__(self, units, last_linear=True, **kwargs):
        super().__init__(**kwargs)
        layers = []
        n = len(units)
        for i, unit in enumerate(units):
            layers.append(Linear(unit))
            if i < n - 1 or not last_linear:
                layers.append(ReLU())
        self._layers = layers
        
    @tf.function
    def call(self, x):
        for layer in self._layers:
            x = layer(x)
        return x

model = NeuralNetwork([3, 1])
for iteration in range(1001):
    loss = train_step(model)
    
print('Mean absolute error is: ', tf.reduce_mean(tf.abs(y - model(x))).numpy())
pprint(model.variables)
reduced_model = reduce(tf.linalg.matmul, model.variables)
print(reduced_model)
print(tf.reduce_any(tf.abs(model(x) - x @ reduced_model) < 1e-6))

Mean absolute error is:  0.00068460405
[<tf.Variable 'linear_19/Variable:0' shape=(5, 3) dtype=float32, numpy=
array([[-0.03742151, -0.5278314 ,  0.31391507],
       [-0.13208945,  0.16571082,  0.5277416 ],
       [ 0.4234567 ,  0.6163233 ,  0.4917594 ],
       [ 1.0218755 ,  0.04149501,  0.99756354],
       [ 0.03784864,  0.9945738 ,  1.5945246 ]], dtype=float32)>,
 <tf.Variable 'linear_20/Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[1.0926309],
       [1.0222646],
       [1.8456209]], dtype=float32)>]
tf.Tensor(
[[-1.1029840e-03]
 [ 9.9908626e-01]
 [ 2.0003288e+00]
 [ 3.0000756e+00]
 [ 4.0009604e+00]], shape=(5, 1), dtype=float32)
tf.Tensor(True, shape=(), dtype=bool)


With our underline ground truth to be a simple linear model, the added non-linear
activations is not really helpful, and in fact it made the optimization harder.

Since activation functions usually follows immediately after linear transformations, we can
fuse them together, so that the model code can be simpler.

In [77]:
class Linear(tf.keras.layers.Layer):
    def __init__(self, units, use_bias=True, activation='linear', **kwargs):
        super(Linear, self).__init__(**kwargs)
        self.units = units
        self.use_bias = use_bias
        self.activation = activation
    
    def build(self, input_shape):
        self._weights = self.add_weight(shape=(input_shape[-1], self.units))
        if self.use_bias:
            self._bias = self.add_weight(shape=(self.units), initializer='ones')
        super().build(input_shape)
    
    @tf.function
    def call(self, x):
        output = tf.linalg.matmul(x, self._weights)
        if self.use_bias:
            output += self._bias
        if self.activation == 'relu':
            output = tf.maximum(tf.constant(0, x.dtype), output)
        return output

class NeuralNetwork(tf.keras.Model):
    def __init__(self, units, use_bias=True, last_linear=True, **kwargs):
        super().__init__(**kwargs)
        layers = [Linear(unit, use_bias, 'relu') for unit in units[:-1]]
        layers.append(Linear(units[-1], use_bias, 'linear' if last_linear else 'relu'))
        self._layers = layers
    
    @tf.function
    def call(self, x):
        for layer in self._layers:
            x = layer(x)
        return x

model = NeuralNetwork([3, 1])
for iteration in range(1001):
    loss = train_step(model)
print('Mean absolute error is: ', tf.reduce_mean(tf.abs(y - model(x))).numpy())
pprint(model.variables)

Mean absolute error is:  0.077715084
[<tf.Variable 'linear_21/Variable:0' shape=(5, 3) dtype=float32, numpy=
array([[ 0.6122507 ,  0.01819468, -0.4076524 ],
       [ 0.20464526,  0.68820965,  0.12365843],
       [-0.02384439,  1.0515662 , -0.0319806 ],
       [-0.43579015,  1.1295365 , -0.6047206 ],
       [-0.9452195 ,  1.3392977 , -0.84022063]], dtype=float32)>,
 <tf.Variable 'linear_21/Variable:0' shape=(3,) dtype=float32, numpy=array([0.90552   , 0.45890853, 1.5429873 ], dtype=float32)>,
 <tf.Variable 'linear_22/Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[-0.7515042],
       [ 1.8287449],
       [-0.9958008]], dtype=float32)>,
 <tf.Variable 'linear_22/Variable:0' shape=(1,) dtype=float32, numpy=array([1.3411213], dtype=float32)>]


### Fully Connected Networks